In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "LeoLM/leo-mistral-hessianai-7b-chat"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from peft import PeftModel
ft_model = PeftModel.from_pretrained(base_model, "textminr/tl-leolm-7b")

adapter_config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

In [11]:
prompt_format = "<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
system_prompt = """Du bist ein Experte im Benennen von Themen. Die wird eine Reihe von Wörtern bereitgestellt und deine Aufgabe ist es, zu diesen Wörtern ein passendes Thema zu finden. Verwende, wenn möglich, maximal drei Wörter. Antworte nur im Format 'Thema: <Thema>'."""
prompt = ""

In [12]:
model_input = eval_tokenizer(prompt_format.format(system_prompt=system_prompt, prompt=prompt), return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


system
Du bist ein Experte im Benennen von Themen. Die wird eine Reihe von Wörtern bereitgestellt und deine Aufgabe ist es, zu diesen Wörtern ein passendes Thema zu finden. Verwende, wenn möglich, maximal drei Wörter. Antworte nur im Format 'Thema: <Thema>'. 
 user
church, archway, building, window, gothic, nave, side, value, tower 
 assistant
Thema: Architektur und Architekturgeschichte 

